#### Defined by User

In [ ]:
## Define the model monitor name here that you will be creating 
MONITOR_NAME = "mm-demo"
## Specify the data source [aws_s3,sql,local]
DATA_SOURCE = "local"
## Specify Training type, [training,retraining]
INPUT_TRAIN_TYPE = "training"

## Specify to True if you are running this notebook outside Dkube
OUTSIDE_DKUBE = False
## If you are running Outside Dkube Fill the Dkube Authentication token , Dkube url, Dkube username below
TOKEN = ""
URL = ""
DKUBEUSERNAME = ""

## if you are running outside Dkube, Fill these AWS_S3 details 
ACCESS_KEY = ""
SECRET_KEY = ""

## if you are running outside Dkube, Fill these SQL details
HOSTNAME = ""
DATABASENAME = ""
DBUSERNAME = ""
PASSWORD = ""

In [ ]:
details = {}
details['MONITOR_NAME'] = MONITOR_NAME
details['DATA_SOURCE'] = DATA_SOURCE
details['INPUT_TRAIN_TYPE'] = INPUT_TRAIN_TYPE
details ['OUTSIDE_DKUBE'] = OUTSIDE_DKUBE
if OUTSIDE_DKUBE:
    details['TOKEN'] = TOKEN
    details['URL'] = URL
    details['DKUBEUSERNAME'] = DKUBEUSERNAME
    if DATA_SOURCE == 'aws_s3':
        details['ACCESS_KEY'] = ACCESS_KEY
        details['SECRET_KEY'] = SECRET_KEY
    if DATA_SOURCE == 'sql':
        details['HOSTNAME'] = HOSTNAME
        details['DATABASENAME'] = DATABASENAME
        details['DBUSERNAME'] = DBUSERNAME
        details['PASSWORD'] = PASSWORD

#### Dkube Resources

In [ ]:
import os, time,json,shutil
from dkube.sdk import *

In [ ]:
if OUTSIDE_DKUBE:
    token = TOKEN
    dkube_url = URL
    username = DKUBEUSERNAME
    access_key = ACCESS_KEY
    secret_key = SECRET_KEY
else:
    token = os.getenv("DKUBE_USER_ACCESS_TOKEN")
    username =  os.getenv("USER")
    dkube_url = os.getenv("DKUBE_URL")
    access_key = os.getenv("AWS_ACCESS_KEY_ID")
    secret_key = os.getenv("AWS_SECRET_ACCESS_KEY")


api = DkubeApi(URL=dkube_url,token=token)
if username == api.validate_token()['username']:
    pass
else:
    print("Invalid User, please check your username, token details first")

#### Code

In [ ]:
try:
    code = DkubeCode(username, name="insurance")
    code.update_git_details("https://github.com/oneconvergence/dkube-examples.git",branch="monitoring")
    api.create_code(code)

except Exception as e:
    if e.reason.lower()!="conflict":
        response = e.body
        print(f"Failed[{response.code}]: {response.message}")

#### Dataset

In [ ]:
if DATA_SOURCE == 'local' or OUTSIDE_DKUBE :
    try:
        dataset = DkubeDataset(username, name='insurance-data')
        dataset.update_git_details(url="https://dkube-examples-data.s3.us-west-2.amazonaws.com/monitoring-insurance/training-data/insurance.csv")
        dataset.update_dataset_source(source="pub_url")
        api.create_dataset(dataset)

    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
            
            
if DATA_SOURCE == 'aws_s3' and not OUTSIDE_DKUBE:
    try:
        dataset = DkubeDataset(username, name='insurance-data',remote=True)
        dataset.update_dataset_source('aws_s3')
        dataset.update_awss3_details(bucket="dkube-examples-data",prefix="monitoring-insurance/training-data",key=access_key,secret=secret_key)
        api.create_dataset(dataset)

    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")   

### Model Monitor Datasets

##### (Training / Retraining) Dataset

In [ ]:
if not OUTSIDE_DKUBE:
    try:
        dataset = DkubeDataset(username, name='insurance-training-data')
        api.create_dataset(dataset)

    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")

##### Predict Dataset

In [ ]:
pname =  MONITOR_NAME+'-predict'
prefix = MONITOR_NAME+'/predict'

if DATA_SOURCE == 'local':
    try:
        dataset = DkubeDataset(username, name=pname)
        api.create_dataset(dataset)
    
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
            

if DATA_SOURCE == 'aws_s3':
    try:
        dataset = DkubeDataset(username, name=pname,remote=True)
        dataset.update_dataset_source('aws_s3')
        dataset.update_awss3_details(bucket="mm-workflow",prefix=prefix,key=access_key,secret=secret_key)
        api.create_dataset(dataset)
        
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
    

##### Labelled Dataset

In [ ]:
gname = MONITOR_NAME+'-groundtruth'
prefix = MONITOR_NAME+'/groundtruth'

if DATA_SOURCE == 'local':
    try:
        dataset = DkubeDataset(username, name=gname)
        api.create_dataset(dataset)
    
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
            

if DATA_SOURCE == 'aws_s3':
    try:
        dataset = DkubeDataset(username, name=gname,remote=True)
        dataset.update_dataset_source('aws_s3')
        dataset.update_awss3_details(bucket="mm-workflow",prefix=prefix,key=access_key,secret=secret_key)
        api.create_dataset(dataset)
        
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")

#### Model

In [ ]:
if api.dkubeinfo['features'] == ['monitoring']:
    try:
        model = DkubeModel(username, name='insurance-model')
        model.update_model_source(source='git')
        model.update_git_details('https://github.com/oneconvergence/dkube-examples/blob/monitoring/insurance/model.joblib',branch="monitoring")
        api.create_model(model)

    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")
else:
    try:
        model = DkubeModel(username, name='insurance-model')
        model.update_model_source(source='dvs')
        api.create_model(model)
    
    except Exception as e:
        if e.reason.lower()!="conflict":
            response = e.body
            print(f"Failed[{response.code}]: {response.message}")



#### Saving MM-Details

In [ ]:
HOME = '/home/'+username+'/workspace/insurance/insurance'

In [ ]:
with open("details.txt","w") as f:
    json.dump(details,f)

In [ ]:
if not OUTSIDE_DKUBE:
    shutil.copy('details.txt',HOME)

#### Cleanup

In [ ]:
## Set CLEANUP = True, after your experiment is complete.
CLEANUP = False
if CLEANUP:
    api.delete_code(username,'insurance',force=True)
    api.delete_dataset(username,'insurance-data',force=True)
    api.delete_dataset(username,'insurance-training-data',force=True)
    api.delete_dataset(username,pname,force=True)
    api.delete_dataset(username,gname,force=True)
    api.delete_model(username,'insurance-model',force=True)
    